In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from google.colab import files

# --- Upload photo ---
uploaded = files.upload()
image_filename = list(uploaded.keys())[0]

# --- Read image ---
image = cv2.imread(image_filename)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# --- Detect red points (used for flattening surface) ---
# Red in BGR: high R, low G and B
mask_red = (image[:, :, 2] > 150) & (image[:, :, 1] < 80) & (image[:, :, 0] < 80)
red_points = np.argwhere(mask_red)

if len(red_points) != 4:
    raise ValueError(f"Expected 4 red points, found {len(red_points)}. Ensure exactly 4 red markers exist.")

# Sort red points: top-left, top-right, bottom-right, bottom-left
red_sorted = sorted(red_points, key=lambda p: (p[0], p[1]))
top = sorted(red_sorted[:2], key=lambda p: p[1])
bottom = sorted(red_sorted[2:], key=lambda p: p[1])
pts_src = np.array([top[0][::-1], top[1][::-1], bottom[1][::-1], bottom[0][::-1]], dtype=np.float32)

# --- Detect yellow pixels robustly using HSV ---
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
lower_yellow = np.array([20, 100, 100])
upper_yellow = np.array([35, 255, 255])
mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
yellow_points = np.argwhere(mask_yellow > 0)

if len(yellow_points) < 4:
    raise ValueError(f"Expected at least 4 yellow edge points, found {len(yellow_points)}.")

# Use bounding rectangle of yellow points as destination rectangle
y_min, x_min = yellow_points.min(axis=0)
y_max, x_max = yellow_points.max(axis=0)
pts_dst = np.array([
    [x_min, y_min],
    [x_max, y_min],
    [x_max, y_max],
    [x_min, y_max]
], dtype=np.float32)

# --- Compute homography and warp ---
H, _ = cv2.findHomography(pts_src, pts_dst)
warped = cv2.warpPerspective(image_rgb, H, (image.shape[1], image.shape[0]))

# --- Show results ---
plt.figure(figsize=(12,6))

plt.subplot(1,2,1)
plt.imshow(image_rgb)
plt.scatter([p[1] for p in red_points], [p[0] for p in red_points], c='red', s=80, label='Red Points')
plt.scatter([p[1] for p in yellow_points], [p[0] for p in yellow_points], c='yellow', s=20, label='Yellow Edges')
plt.title("Original Image")
plt.legend()

plt.subplot(1,2,2)
plt.imshow(warped)
plt.title("Warped Image (Red → Square, Yellow → Rectangle)")
plt.show()